In [19]:
import pandas as pd
import numpy as np
import json
import ast
import re
import nltk
import spacy

from nltk import pos_tag, word_tokenize
from collections import Counter


In [2]:
# importing allrecipes data and converting to dataframe
with open('scraping/recipes/recipes_allrecipes.json', 'r') as file:
    ar_data = json.load(file)
ar_data = pd.DataFrame(ar_data)

ar_data[['title', 'category','rating','rating_count']] = ar_data['basic_info'].apply(pd.Series)
ar_data = ar_data.drop(columns=['basic_info']) # removing old column

ar_data[['cook_time', 'total_time', 'yield', 'servings']] = ar_data['prep_data'].apply(
    lambda x: pd.Series({
        'cook_time': x.get('cook_time', ''),
        'total_time': x.get('total_time', ''),
        'yield': x.get('yield', ''),
        'servings': x.get('servings', '')
    }))
ar_data = ar_data.drop(columns=['prep_data'])

ar_data[['calories','fat','carbs','protein']] = ar_data['nutritions'].apply(pd.Series)
ar_data = ar_data.drop(columns=['nutritions'])


ar_data

,ingredients,directions,title,category,rating,rating_count,cook_time,total_time,yield,servings,calories,fat,carbs,protein
0,"[2 tablespoons extra-virgin olive oil, divided...",[Preheat the oven to 350 degrees F (175 degree...,Lower-Carb Healthy Vegetable Casserole,Recipes,5.0,5,35 mins,55 mins,8 servings,8,126,9g,7g,5g
1,"[¼ cup sliced fresh strawberries, 1 ½ fluid ou...","[Place strawberries, tequila, lime juice, trip...",Classic Frozen Strawberry Margarita,Recipes,4.9,16,,10 mins,,1,206,0g,21g,0g
2,[],[],15 Best Ramadan Recipes,Recipes,N/A,N/A,,,,,NaN,NaN,NaN,NaN
3,[1 ½ cups warm water (110 to 115 degrees F/43 ...,"[Combine water, honey, and yeast in a stand mi...",Golden Egg Loaves (or Braids),Recipes,4.3,3,25 mins,2 hrs 55 mins,2 loaves,24,186,5g,30g,6g
4,"[1 cup chocolate graham crackers, ¼ cup white ...",[Preheat the oven to 375 degrees F (190 degree...,Chocolate Graham Crust,Desserts,4.0,1,10 mins,50 mins,1 pie crust,8,103,7g,11g,1g
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
21572,"[1 ⅓ cups French-fried onions, crushed, 1 larg...","[Gather all ingredients., Dotdash Meredith Foo...",Crunchy French Onion Chicken,Recipes,4.5,339,20 mins,30 mins,,4,623,40g,32g,28g
21573,"[2 summer squash, ends trimmed, ¼ cup olive o...",[Preheat the oven to 450 degrees F (230 degree...,Roasted Summer Squash,Side Dish,4.7,71,5 mins,20 mins,,4,139,14g,4g,1g
21574,"[1 (8 ounce) container ricotta cheese, 2 large...","[Gather all ingredients., Dotdash Meredith Foo...",Ricotta Gnocchi,Recipes,4.5,712,15 mins,1 hr,,5,442,26g,27g,22g
21575,"[2 cups sliced fresh peaches, 2 cups sliced fr...",[Preheat oven to 350 degrees F (175 degrees C)...,Fresh Fruit Basket Cobbler,Recipes,4.5,2,20 mins,1 hr,1 8x13-inch dish,8,358,8g,69g,5g


In [3]:
# importing bon appetit data and converting to dataframe
with open('scraping/recipes/recipes_bon_appetit.json', 'r') as file:
    ba_data = json.load(file)
ba_data = pd.DataFrame(ba_data)

ba_data[['title', 'rating']] = ba_data['basic_info'].apply(pd.Series)
ba_data = ba_data.drop(columns=['basic_info']) # removing old column

ba_data[['total_time', 'servings']] = ba_data['prep_data'].apply(pd.Series)
ba_data = ba_data.drop(columns=['prep_data']) # removing old column

ba_data[['calories', 'fat', 'saturated_fat', 'cholesterol','carbs', 'dietary_fiber', 
         'total_sugars', 'protein', 'sodium']] = ba_data['nutritions'].apply(pd.Series)
ba_data = ba_data.drop(columns=['nutritions']) # removing old column
ba_data = ba_data.drop(columns=['saturated_fat','cholesterol','dietary_fiber','total_sugars','sodium'])  # removing unneded nutrition info

ba_data

,ingredients,directions,title,rating,total_time,servings,calories,fat,carbs,protein
0,"[½ cup (63 g) all-purpose flour, ¼ cup (50 g) ...","[Streusel - Step 1: Whisk , , , and in a mediu...",Butternut Squash Coffee Cake,4.0,1 hour 20 minutes,8–12 servings,None,None,None,None
1,"[4 large red plums (about 1 lb.), cut into ½""-...","[None - Step 1: Bring , , , , and ¼ cup water ...",Jammy Cumin Plums and Feta Salad,4.7,1 hour,4 servings,None,None,None,None
2,"[2 heads of Little Gem lettuce, trimmed, quart...","[None - Step 1: Place , in a large bowl and ch...",Cheese-Cracker-Crusted Chicken,4.8,NaN,4 servings,None,None,None,None
3,"[3 large egg yolks, 4 oz. Pecorino Romano, fin...","[None - Step 1: Whisk and , in a medium bowl t...","Easy, Cheesy, Almost-Carbonara Udon",5.0,30 minutes,4 servings,None,None,None,None
4,"[2 large eggs, 10 Tbsp. unsalted butter, melte...",[None - Step 1: Preheat oven to 375°. Line a s...,"Butternut Squash, Coconut, and Ginger Muffins",4.4,NaN,Makes 1 dozen,None,None,None,None
...,...,...,...,...,...,...,...,...,...,...
2399,"[1¼ cups (156 g) all-purpose flour, ½ tsp. bak...",[None - Step 1: Whisk and in a small bowl to c...,Miso White Chocolate–Macadamia Cookies,5.0,NaN,Makes 10–12,None,None,None,None
2400,"[¾ cup raw pecans, 4 oz. cream cheese, room te...",[None - Step 1: Preheat oven to 400°. Toast on...,Classic Cheese Ball,N/A,20 minutes (plus 1–1½ hours for chilling),6–8 servings,None,None,None,None
2401,"[2 Tbsp. extra-virgin olive oil, 1 tsp. ajwain...",[None - Step 1: Heat oil in a large skillet ov...,Bhindi Ki Sabzi,5.0,NaN,2–4 servings,None,None,None,None
2402,"[1½ cups pecans, 2 garlic cloves, finely grate...",[None - Step 1: Preheat oven to 350°. Toast pe...,Kale Salad With Pecan Vinaigrette,4.8,NaN,8 Servings,None,None,None,None


In [4]:
combined_data = pd.concat([ba_data, ar_data], ignore_index=True)

In [5]:
def count_ingredients(ingredients_list: list) -> int:
    '''    
    Count the number of ingredients in a given recipe.

    Arguments:
    ingredients_list (list): A list of ingredient strings.

    Returns:
    int: The count of non-empty ingredient strings. Returns 0 if the input list is empty.'''
    
    if ingredients_list:  # Check if the list is not empty
        # Counting non-empty strings in the list
        return sum(1 for ingredient in ingredients_list if ingredient.strip())  
    return 0  # Return 0 for empty lists

In [6]:
# adding ingredient count to dataframe
combined_data['ingredient_count'] = combined_data['ingredients'].apply(count_ingredients)
combined_data.head(3)

,ingredients,directions,title,rating,total_time,servings,calories,fat,carbs,protein,category,rating_count,cook_time,yield,ingredient_count
0,"[½ cup (63 g) all-purpose flour, ¼ cup (50 g) ...","[Streusel - Step 1: Whisk , , , and in a mediu...",Butternut Squash Coffee Cake,4.0,1 hour 20 minutes,8–12 servings,None,None,None,None,NaN,NaN,NaN,NaN,18
1,"[4 large red plums (about 1 lb.), cut into ½""-...","[None - Step 1: Bring , , , , and ¼ cup water ...",Jammy Cumin Plums and Feta Salad,4.7,1 hour,4 servings,None,None,None,None,NaN,NaN,NaN,NaN,8
2,"[2 heads of Little Gem lettuce, trimmed, quart...","[None - Step 1: Place , in a large bowl and ch...",Cheese-Cracker-Crusted Chicken,4.8,NaN,4 servings,None,None,None,None,NaN,NaN,NaN,NaN,13


In [24]:
combined_data.to_csv('data/recipe_data.csv', index=False)